In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import json
import time

def extract_polygons_with_selenium(url):
    """Extract polygon data and calculate area using Selenium and Turf.js"""
    
    # Configure Chrome options
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Run in background
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    
    driver = webdriver.Chrome(options=chrome_options)
    
    try:
        print("Loading webpage...")
        driver.get(url)
        
        # Wait for the map to load by waiting for the "Please wait" message to disappear
        WebDriverWait(driver, 60).until(
            EC.invisibility_of_element_located((By.ID, "Pleasewait"))
        )
        
        print("Map loaded. Injecting Turf.js for area calculation...")
        
        # Inject the Turf.js library into the page
        driver.execute_script(
            "var script = document.createElement('script');"
            "script.src = 'https://npmcdn.com/@turf/turf/turf.min.js';"
            "document.head.appendChild(script);"
        )
        # Give the script a moment to load
        time.sleep(2)

        print("Extracting polygon data and calculating area...")
        
        # Execute JavaScript to extract polygon data and calculate area
        extract_script = """
        var polygonData = [];
        
        // Helper function to process each layer
        function processLayer(layer, type) {
            if (layer instanceof L.Polygon) {
                var coordinates = layer.getLatLngs()[0].map(function(latlng) {
                    return [latlng.lng, latlng.lat];
                });

                // Ensure the polygon is closed for valid GeoJSON
                if (coordinates.length > 0 && (coordinates[0][0] !== coordinates[coordinates.length - 1][0] || coordinates[0][1] !== coordinates[coordinates.length - 1][1])) {
                    coordinates.push(coordinates[0]);
                }

                var polygonGeoJSON = {
                    "type": "Polygon",
                    "coordinates": [coordinates]
                };
                
                var areaInSqMeters = turf.area(polygonGeoJSON);

                polygonData.push({
                    type: type,
                    surveyNo: layer.SurveyNo || '',
                    vlgCode: layer.Vlgcode || layer.VlgLgdcode || '',
                    surnoc: layer.Surnoc || '',
                    hissa: layer.Hissa || '',
                    coordinates: coordinates,
                    area: areaInSqMeters
                });
            }
        }

        // Extract Survey Boundary polygons
        if (typeof SurveyBoundLyr !== 'undefined') {
            SurveyBoundLyr.eachLayer(function(layer) {
                processLayer(layer, 'survey');
            });
        }
        
        // Extract Hissa Boundary polygons
        if (typeof HissaBoundLyr !== 'undefined') {
            HissaBoundLyr.eachLayer(function(layer) {
                processLayer(layer, 'hissa');
            });
        }
        
        return polygonData;
        """
        
        # Execute the script and get results
        polygon_data = driver.execute_script(extract_script)
        
        print(f"Extracted {len(polygon_data)} polygons with their areas.")
        
        return polygon_data
        
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    finally:
        driver.quit()

def save_selenium_data(polygon_data, filename_prefix):
    """Save extracted polygon data, including area, to GeoJSON files"""
    
    if not polygon_data:
        print("No data was extracted to save.")
        return
    
    # Separate survey and hissa polygons
    survey_polygons = [p for p in polygon_data if p['type'] == 'survey']
    hissa_polygons = [p for p in polygon_data if p['type'] == 'hissa']
    
    # Save as GeoJSON
    for poly_type, polygons in [('survey', survey_polygons), ('hissa', hissa_polygons)]:
        if not polygons:
            continue
            
        features = []
        for poly in polygons:
            feature = {
                "type": "Feature",
                "properties": {
                    "type": poly['type'],
                    "surveyNo": poly['surveyNo'],
                    "vlgCode": poly['vlgCode'],
                    "surnoc": poly['surnoc'],
                    "hissa": poly['hissa'],
                    "area_sq_meters": round(poly.get('area', 0), 2)  # Add area to properties
                },
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [poly['coordinates']]
                }
            }
            features.append(feature)
        
        geojson = {
            "type": "FeatureCollection",
            "features": features
        }
        
        # A single file for both types for simplicity, or name them uniquely.
        filename = f"{filename_prefix}_polygons.geojson"
        with open(filename, 'w') as f:
            json.dump(geojson, f, indent=2)
        
        print(f"Successfully saved {len(features)} polygons to {filename}")

# --- Main Execution ---
if __name__ == "__main__":
    # The URL for the map to be scraped
    webpage_url = "https://rdservices.karnataka.gov.in/service84/Default/WebMaps?pData=415073525330335242764D75314C38683631454B2F673D3D"
    
    extracted_data = extract_polygons_with_selenium(webpage_url)
    
    if extracted_data:
        save_selenium_data(extracted_data, "panathur_extracted")
    else:
        print("Polygon data extraction failed. No files were saved.")

Loading webpage...
Map loaded. Injecting Turf.js for area calculation...
Extracting polygon data and calculating area...
Extracted 168 polygons with their areas.
Successfully saved 168 polygons to panathur_extracted_polygons.geojson
